In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import random
import os
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, lupus_constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The Data

In [3]:
# df = pd.read_csv('../../data/more_features/more_feats_correlated_0.1.csv')
# df = df.fillna(-1)
# df.head()

In [4]:
#train_df = pd.read_csv('../../final/data/train_set_basic.csv')
#train_df.head()

In [5]:
#train_df.iloc[0]

In [277]:
# test_df = pd.read_csv('../../final/data/test_set_constant.csv')
# test_df = pd.read_csv('../../lupus_trial/data/anemia/test_set_constant.csv')
# test_df = pd.read_csv('../../../anemia_ml4hc/data/test_set_constant.csv')
test_df = pd.read_csv('../../../anemia_ml4hc/data/val_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,9.496631,-1.000000,-1.000000,3.515439,375.685261,101.027779,122.174205,2.820006,1,1.276220,147.925454,74.188240,23.800042,11.545421,97.593498,28.489894,32.520362,1
1,7.092780,336.562609,-1.000000,-1.000000,115.507513,75.569193,35.449822,2.815743,0,0.870413,105.709466,118.836405,30.607322,2.113748,88.833122,21.278341,30.690491,3
2,6.554136,-1.000000,1.774495,6.145810,-1.000000,86.849700,-1.000000,2.263958,0,0.569184,139.078814,-1.000000,-1.000000,-1.000000,-1.000000,19.662409,-1.000000,6
3,12.417159,252.249921,1.701860,0.188413,515.449324,76.102948,-1.000000,4.894880,1,0.576132,4.588230,-1.000000,-1.000000,-1.000000,-1.000000,37.251478,-1.000000,3
4,7.748672,258.549935,2.980693,5.137341,363.214987,82.395181,99.002425,2.821283,1,0.885522,84.419465,33.706997,19.591641,27.675132,69.578682,23.246016,27.257252,5


In [278]:
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((5600, 17), (5600,))

In [279]:
# utils.get_dt_performance(df)

In [280]:
# df.label.value_counts()

In [281]:
# class_dict = constants.CLASS_DICT
# df['label'] = df['label'].replace(class_dict)
# X = df.iloc[:, 0:-1]
# y = df.iloc[:, -1]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
# X_train, y_train = np.array(X_train), np.array(y_train)
# X_test, y_test = np.array(X_test), np.array(y_test)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [282]:
# y_test[:5]

In [283]:
# action_list = list(class_dict) + [col  for col in df.columns if col!='label']
# action_list

In [284]:
# len(action_list)

#### Testing

In [285]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/constant_test_set/dqn_missing_1_6000000', 
#                             training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [315]:
# dqn_model = utils.load_dqn3('../../../anemia_ml4hc/models/sb/dueling_double_dqn/dddqn_basic_3000000_steps.zip')
dqn_model = utils.load_dqn(f'../../../anemia_ml4hc/models/sb/missingness/double_dqn_pr_noisiness_2_12000000.pkl')
# dqn_model = utils.load_dqn3('../../final/models/dqn_noisy_6_missing_3_21300000')
test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
test_df.head()

Using just stable baselines (not 3)
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Testing done.....


,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success
0,0.0,5.0,1.0,1.0,1.0,"[gender, hemoglobin, rbc, segmented_neutrophil...",0.0,1.0
1,1.0,6.0,1.0,3.0,3.0,"[gender, hemoglobin, rbc, ferritin, mcv, Anemi...",0.0,1.0
2,2.0,5.0,1.0,6.0,6.0,"[gender, hemoglobin, rbc, ret_count, Aplastic ...",0.0,1.0
3,3.0,6.0,1.0,3.0,3.0,"[gender, hemoglobin, rbc, ferritin, mcv, Anemi...",0.0,1.0
4,4.0,5.0,1.0,5.0,5.0,"[gender, hemoglobin, rbc, ret_count, Hemolytic...",0.0,1.0


In [316]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

93.58928571428572

In [317]:
test_df[test_df.episode_length==test_df.episode_length.min()].iloc[-1]['trajectory']

['gender', 'hemoglobin', 'No anemia']

In [318]:
test_df.episode_length.min(), test_df.episode_length.max()

(3.0, 7.0)

In [319]:
# test_df[test_df.episode_length == test_df.episode_length.max()]

In [320]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(4.8653571428571425, 0.7021428571428572)

In [321]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(93.58928571428572, 93.26442456734509, 96.24146742302162)

In [322]:
test_df.y_pred.value_counts()

4.0    788
1.0    780
0.0    763
2.0    719
5.0    707
6.0    699
3.0    577
7.0    567
Name: y_pred, dtype: int64

#### Saving files

In [ ]:
# test_df.to_csv(f'../../lupus_trial/test_dfs/orig/dqn_test_df_9500000.csv', index=False)
# success_df.to_csv(f'../../lupus_trial/test_dfs/orig/dqn_success_df_9500000.csv', index=False)

In [ ]:
# for steps in [int(9.5e6), int(10e6), int(10.5e6), int(11e6), int(11.5e6), int(12e6), int(12.5e6), int(13e6), int(13.5e6), int(15e6), int(16e6), int(17e6), int(18e6), int(19e6), int(20e6), int(21e6), int(22e6), int(23e6), int(24e6)]:
#     training_env = utils.create_env(X_train, y_train)
#     dqn_model = utils.load_dqn3(f'../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_6_{steps}', 
#                                 training_env)
#     test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
#     success_rate, success_df = utils.success_rate(test_df)
#     avg_length, avg_return = utils.get_avg_length_reward(test_df)
#     print(f'steps: {steps}, success_rate: {success_rate}, avg_length: {avg_length}, avg_return: {avg_return}')

#### Confusion matrix and classification report

In [ ]:
# test_df = pd.read_csv('../../test_dfs/many_features/0.1/correlated/test_df3_missing3_12500000.csv')
# test_df.head()

In [ ]:
utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# def plot_confusion_matrix(y_actual, y_pred, save=False, filename=False):
#     from sklearn.metrics import confusion_matrix
#     cm = confusion_matrix(y_actual, y_pred)
#     cm_df = pd.DataFrame(cm, index = [0, 1, 2, 3, 4, 5, 6], columns = [0, 1, 2, 3, 4, 5, 6], dtype='object')
#     #cm_df = pd.DataFrame(cm, index = constants.CLASS_DICT.keys(), columns = constants.CLASS_DICT.keys())
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm_df, annot=True)
#     plt.title('Confusion Matrix')
#     plt.ylabel('Actual Anemia')
#     plt.xlabel('Predicted Anemia')
#     plt.tight_layout()
#     if save:
#         plt.savefig(filename)
#     plt.show()
#     plt.close()

In [ ]:
utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])